In [2]:
import pandas as pd
import numpy as np

## GroupBy机制


+    使用一个或多个键（形式可以是函数、数组或DataFrame列名）分割pandas对象。
+   计算分组的概述统计，比如数量、平均值或标准差，或是用户定义的函数。
+  应用组内转换或其他运算，如规格化、线性回归、排名或选取子集等。
+ 计算透视表或交叉表。
+ 执行分位数分析以及其它统计分组分析。


### GroupBy的工作原理
1. 拆分(split)：根据传入的分组方式进行分组
2. 应用(apply)：对每个分组运用对应的函数
3. 合并(combine)：合并数据集

![](https://camo.githubusercontent.com/8e5549c8c55780a51555d0c193f8568e73796143/687474703a2f2f75706c6f61642d696d616765732e6a69616e7368752e696f2f75706c6f61645f696d616765732f373137383639312d653563363731653039656366393462652e706e673f696d6167654d6f6772322f6175746f2d6f7269656e742f7374726970253743696d61676556696577322f322f772f31323430)

分组键可以有很多中形式， 而且类型可以不相同
+ 列表或者数组，其长度与待分组的轴一样
+ DataFrame的每个列名值
+ 字典或者Series，给出待分组轴上的值与分组名之间对应关系
+ 函数，用于处理轴索引或者索引中的各个标签

### 分组操作

In [3]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                       'key2' : ['one', 'two', 'one', 'two', 'one'],
                       'data1' : np.random.randn(5),
                       'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,1.341073,2.355754
1,a,two,-1.385922,0.466002
2,b,one,-2.474644,1.456787
3,b,two,0.654396,1.274367
4,a,one,-1.891275,1.906919


示例1：按key1进行分组，并计算data1列的平均值

In [8]:
grouped = df['data1'].groupby(df['key1'])
grouped

对`DataFrame`进行分组，返回一个GroupBy对象

In [9]:
grouped.mean()

key1
a   -0.645375
b   -0.910124
Name: data1, dtype: float64

示例2：根据key1，key2进行分组，并计算data1的平均值

In [10]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one    -0.275101
      two    -1.385922
b     one    -2.474644
      two     0.654396
Name: data1, dtype: float64

In [11]:
means.unstack()

key2,one,two
key1,,
a,-0.275101,-1.385922
b,-2.474644,0.654396


示例3：根据适当长度的数组进行分组


In [15]:
# df['data1'].size  5
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005   -1.385922
            2006   -2.474644
Ohio        2005    0.997735
            2006   -1.891275
Name: data1, dtype: float64

In [26]:
# 上面代码可以看出如下：
df2 = pd.concat([df['data1'],
                 pd.Series(states, name='states'),
                 pd.Series(years, name='years')],
                axis=1)
df2['data1'].groupby([df2['states'], df2['years']]).mean()

states      years
California  2005    -1.385922
            2006    -2.474644
Ohio        2005     0.997735
            2006    -1.891275
Name: data1, dtype: float64

示例4：根据df的key1进行分组，并计算平均值

In [27]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.645375,1.576225
b,-0.910124,1.365577


**注意：**不相关的数据(非数组型数列)会被过滤掉。（如，上面的key2）

示例5：根据df的key1, key2进行分组，并计算平均值

In [29]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -0.275101  2.131336
     two  -1.385922  0.466002
b    one  -2.474644  1.456787
     two   0.654396  1.274367

示例5：根据df的key1，key2进行分组，并计算每个小组里面数组的个数

In [31]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### 对分组进行迭代

GroupBy对象支持迭代，可以产生一组二元元组
GroupBy--->(name1, group1),(name2, group2)....

示例1：根据df的key1进行分组，并遍历分组产生的GroupBy对象

In [32]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  1.341073  2.355754
1    a  two -1.385922  0.466002
4    a  one -1.891275  1.906919
b
  key1 key2     data1     data2
2    b  one -2.474644  1.456787
3    b  two  0.654396  1.274367


示例1：根据df的key1,key2进行分组，并遍历分组产生的GroupBy对象.  
（这里有点特殊，因为根据多个键值进行分组，返回的name是一个多元元组）


In [35]:
# for name, group in df.groupby(['key1', 'key2']):
for (key1, key2), group in df.groupby(['key1', 'key2']):
    print(key1, key2)
    print(group)

a one
  key1 key2     data1     data2
0    a  one  1.341073  2.355754
4    a  one -1.891275  1.906919
a two
  key1 key2     data1     data2
1    a  two -1.385922  0.466002
b one
  key1 key2     data1     data2
2    b  one -2.474644  1.456787
b two
  key1 key2     data1     data2
3    b  two  0.654396  1.274367


 示例3：根据df的key1进行分组，并将数据片段变为一个字典

In [36]:
pieces = dict(list(df.groupby('key1')))
pieces

{'a':   key1 key2     data1     data2
 0    a  one  1.341073  2.355754
 1    a  two -1.385922  0.466002
 4    a  one -1.891275  1.906919, 'b':   key1 key2     data1     data2
 2    b  one -2.474644  1.456787
 3    b  two  0.654396  1.274367}

In [37]:
pieces['b']

,key1,key2,data1,data2
2,b,one,-2.474644,1.456787
3,b,two,0.654396,1.274367


示例4：根据df的dtype进行分类(**注意：**可以通过axis=1分类)


In [39]:
df.dtypes


key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [45]:
grouped = df.groupby(df.dtypes, axis=1)
grouped.size()

float64    2
object     2
dtype: int64

In [46]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0  1.341073  2.355754
1 -1.385922  0.466002
2 -2.474644  1.456787
3  0.654396  1.274367
4 -1.891275  1.906919
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### 选取一列或者列的子集

示例1：根据df的key1进行分组，并选取其中的某一列

In [52]:
%%timeit
df.groupby('key1')['data1']

154 µs ± 3.49 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [49]:
df.groupby('key1')[['data2']]

上面两行代码相当于

In [57]:
df['data1'].groupby(df['key1'])

In [60]:
df[['data2']].groupby(df['key1'])

示例2：根据df的key1，key2进行分组，选取其中的data2进行求平均，返回一个DataFrame

In [62]:
df.groupby(['key1', 'key2'])[['data1']].mean()

data1
key1 key2          
a    one  -0.275101
     two  -1.385922
b    one  -2.474644
     two   0.654396

示例3：根据df的key1，key2进行分组，选取其中的data2进行求平均，返回一个Series

In [63]:
df.groupby(['key1','key2'])['data1'].mean()

key1  key2
a     one    -0.275101
      two    -1.385922
b     one    -2.474644
      two     0.654396
Name: data1, dtype: float64

### 传入映射字典进行分组

In [64]:
people = pd.DataFrame(np.random.randn(5, 5),
                          columns=['a', 'b', 'c', 'd', 'e'],
                          index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
people

,a,b,c,d,e
Joe,0.522318,-0.340390,-0.659717,1.107929,0.744146
Steve,0.660020,1.161863,-1.165410,0.773331,-0.517003
Wes,-0.673866,NaN,NaN,-1.098838,1.279254
Jim,0.729824,1.138708,-0.597597,1.027640,-0.219603
Travis,1.474624,-0.685122,1.403698,1.502232,1.371654


示例1：假设已知列的分组关系，根据分组计算列的和。  
(**Note：**如果存在未使用的键，程序不会报错)

In [67]:
mapping = {'a':'red','b':'red','c':'blue','d':'blue','e':'red','f':'orange'}
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,0.448212,0.926074
Steve,-0.392080,1.304880
Wes,-1.098838,0.605388
Jim,0.430043,1.648930
Travis,2.905929,2.161155


示例2：传入Series作为映射，并计算每组的数量

In [68]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [69]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### 通过函数进行分组

示例1：根据索引的长度进行分组

In [70]:
people.groupby(len).sum() 

,a,b,c,d,e
3,0.578276,0.798319,-1.257314,1.036730,1.803797
5,0.660020,1.161863,-1.165410,0.773331,-0.517003
6,1.474624,-0.685122,1.403698,1.502232,1.371654


示例2：根据索引最后一个字母进行分组

In [73]:
people.groupby(lambda x:x[-1]).sum()

,a,b,c,d,e
e,1.182338,0.821473,-1.825127,1.881259,0.227144
m,0.729824,1.138708,-0.597597,1.027640,-0.219603
s,0.800758,-0.685122,1.403698,0.403393,2.650908


### 将函数、数组、列表等混合使用

示例1：根据people索引的长度和key_list进行分组

In [75]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).sum()

a         b         c         d         e
3 one -0.151548 -0.340390 -0.659717  0.009090  2.023400
  two  0.729824  1.138708 -0.597597  1.027640 -0.219603
5 one  0.660020  1.161863 -1.165410  0.773331 -0.517003
6 two  1.474624 -0.685122  1.403698  1.502232  1.371654

### 根据索引级别进行分组

In [77]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                     [1, 3, 5, 1, 3]],
                                     names=['cty', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4,5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.640946  0.938213  0.332981 -2.128284  0.074242
1     -1.245719 -1.602921 -0.475766  0.032530 -0.647863
2      1.544264  0.097320 -0.237489 -0.094617 -2.402969
3     -0.917560 -1.276469 -0.481658 -0.786999 -0.801427

In [79]:
hier_df.groupby(level=0, axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 数据聚合

**聚合指的是任何能够从数组产生标量值的数据转换过程。**之前的例子已经用过一些，比如mean、count、min以及sum等。你可能想知道在GroupBy对象上调用mean()时究竟发生了什么。许多常见的聚合运算（如表10-1所示）都有进行优化。然而，除了这些方法，你还可以使用其它的。

![](https://camo.githubusercontent.com/ceb83fe40f3025fa5a8d17aa6b50e74407b6d95a/687474703a2f2f75706c6f61642d696d616765732e6a69616e7368752e696f2f75706c6f61645f696d616765732f373137383639312d626138646535323465303862316236662e706e673f696d6167654d6f6772322f6175746f2d6f7269656e742f7374726970253743696d61676556696577322f322f772f31323430)

### 普通聚合操作

In [80]:
df

,key1,key2,data1,data2
0,a,one,1.341073,2.355754
1,a,two,-1.385922,0.466002
2,b,one,-2.474644,1.456787
3,b,two,0.654396,1.274367
4,a,one,-1.891275,1.906919


示例1：根据df的key1进行分组，并计算data1的上四分位数

In [82]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.75)

key1
a   -0.022425
b   -0.127864
Name: data1, dtype: float64

**Note:**如果想要自己定义聚合函数可以通过`GroupBy.aggregate(func)`实现

示例2：根据df的key1进行分组，并计算分组数据的极差

In [85]:
def func(arr):
    """
    由于分组的聚合操作是将每个小组的每一列当作整体进行聚合操作，
    所以这里用arr接收
    """
    return arr.max() - arr.min()
df.groupby('key1').agg(func)

,data1,data2
key1,,
a,3.232349,1.889752
b,3.129040,0.182420


示例3：根据df的key1进行分组，并对每组进行描述(describe)

In [86]:
df.groupby('key1').describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.645375  1.738772 -1.891275 -1.638599 -1.385922 -0.022425   
b      2.0 -0.910124  2.212565 -2.474644 -1.692384 -0.910124 -0.127864   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.341073   3.0  1.576225  0.987324  0.466002  1.186461  1.906919   
b     0.654396   2.0  1.365577  0.128990  1.274367  1.319972  1.365577   

                          
           75%       max  
key1                      
a     2.131336  2.355754  
b     1.411182  1.456787

***Note:***如果能用优化过的函数(mean,sum等)就用优化过的函数，因为自定义函数会消耗很多的内存

### 面向列的多函数应用

In [89]:
tips = pd.read_csv('examples/tips.csv')
# 添加小费百分比
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


你已经看到，对Series或DataFrame列的聚合运算其实就是使用aggregate（使用自定义函数）或调用诸如mean、std之类的方法。然而，你可能希望对不同的列使用不同的聚合函数，或一次应用多个函数。

先对数据进行分组, 根据(day, smoker)对tips进行分组

In [90]:
grouped = tips.groupby(['day', 'smoker'])

示例1：根据分组，采集其中的tip_pct，求平均值

In [94]:
grouped_pct = grouped[['tip_pct']]
grouped_pct.agg('mean')

tip_pct
day  smoker          
Fri  No      0.151650
     Yes     0.174783
Sat  No      0.158048
     Yes     0.147906
Sun  No      0.160113
     Yes     0.187250
Thur No      0.160298
     Yes     0.163863

示例2：根据分组，采集其中的tip_pct，求平均值, 标准差, 极差(func)

传入一组函数或函数名，得到的DataFrame的列就会以相应的函数命名：

In [98]:
grouped_pct.agg(['mean', 'std', func])

tip_pct                    
                 mean       std      func
day  smoker                              
Fri  No      0.151650  0.028123  0.067349
     Yes     0.174783  0.051293  0.159925
Sat  No      0.158048  0.039767  0.235193
     Yes     0.147906  0.061375  0.290095
Sun  No      0.160113  0.042347  0.193226
     Yes     0.187250  0.154134  0.644685
Thur No      0.160298  0.038774  0.193350
     Yes     0.163863  0.039389  0.151240

示例3：根据分组，采集其中的tip_pct，求平均值, 标准差, 极差(func)。并为此起别名  
**Note:**可以通过传入列表元组起别名[(name, func), (name,func)...]

In [105]:
grouped_pct.agg([('foo','mean'), ('bar',np.std), ('range',func)])

tip_pct                    
                  foo       bar     range
day  smoker                              
Fri  No      0.151650  0.028123  0.067349
     Yes     0.174783  0.051293  0.159925
Sat  No      0.158048  0.039767  0.235193
     Yes     0.147906  0.061375  0.290095
Sun  No      0.160113  0.042347  0.193226
     Yes     0.187250  0.154134  0.644685
Thur No      0.160298  0.038774  0.193350
     Yes     0.163863  0.039389  0.151240

示例4：根据分组后(grouped)的对象采集tip_pct、total_bill数据，对计算每列数据的统计信息  
（相当于，对每一列进行聚合，然后concat起来）

In [107]:
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

示例5：根据分组后(grouped)的对象采集tip_pct、total_bill数据，对计算每列数据的统计信息。并起别名  

In [108]:
ftuples = [('平均值', 'mean'), ('方差', np.var)]
grouped['tip_pct', 'total_bill'].agg(ftuples)

tip_pct           total_bill            
                  平均值        方差        平均值          方差
day  smoker                                           
Fri  No      0.151650  0.000791  18.420000   25.596333
     Yes     0.174783  0.002631  16.813333   82.562438
Sat  No      0.158048  0.001581  19.661778   79.908965
     Yes     0.147906  0.003767  21.276667  101.387535
Sun  No      0.160113  0.001793  20.506667   66.099980
     Yes     0.187250  0.023757  24.120000  109.046044
Thur No      0.160298  0.001503  17.113111   59.625081
     Yes     0.163863  0.001551  19.190588   69.808518

示例5：根据分组后(grouped)的对象采集tip_pct、total_bill数据，计算tip_pct的个数(count)，total_bill的平均值和方差(mean,var)。并起别名.  
**Note:**可以通过字典完成


In [109]:
map_dict = {
    'tip_pct':'count',
    'total_bill':[('平均值','mean'), ('方差', np.var)]
}
grouped['tip_pct', 'total_bill'].agg(map_dict)

tip_pct total_bill            
              count        平均值          方差
day  smoker                               
Fri  No           4  18.420000   25.596333
     Yes         15  16.813333   82.562438
Sat  No          45  19.661778   79.908965
     Yes         42  21.276667  101.387535
Sun  No          57  20.506667   66.099980
     Yes         19  24.120000  109.046044
Thur No          45  17.113111   59.625081
     Yes         17  19.190588   69.808518